In [ ]:
import scipy.io
import scipy.signal
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit, bisect
from scipy.stats import norm
import numpy as np
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from nptdms import TdmsFile
import csv
from PythonOTtools import callTDMS, flip_molecule, TDMS_reader, gauss, bimodal, fwd_exponential, \
    rev_exponential, covariance_pds, bimodal_fit, find_prelimEvents, timeFilterEvents, plot_prelimEvents_withfilter, \
    cov_mask_fb, callTDMS_fb, find_breakpoints_isoFB, \
    save_events_isoFB, bell_equation

In [ ]:
file_name = 'point to file name here'

#1 for flipped (negative-displacement) molecule, 0 for positive molecule
#Our FB system only really works with positive molecules, so this is mostly not necessary
flipped_molecule = 0
#call the tdms file into a numpy array
trace = callTDMS_fb(file_name)
#flip the molecule
if flipped_molecule:
    trace = flip_molecule(trace)
#plot and visualize the trace
plt.close('all')
plt.figure(figsize=(10, 3))
plt.plot(trace[12], trace[7], label = 'transducer bead pos')
plt.plot(trace[12], trace[8], label = 'motor bead pos')
plt.legend(loc = 'upper right')
plt.show()

In [ ]:
#calculate the covariance and plot it
covar_AB = covariance_pds(trace)
cov_hist = np.histogram(covar_AB, bins = 2000)
plt.plot(cov_hist[1][:-1:], cov_hist[0])
plt.xlim(-10, 60)
plt.show()
plt.close('all')

In [ ]:
#fit to a bimodal to find peaks
peaks = bimodal_fit(trace, covar_AB, a1 = 50000, m1=0, s1=2, a2 = 300000, m2=15, s2 = 8)

In [ ]:
#find prelim events based on covariance threshold
prelimEvents = find_prelimEvents(covar_AB, peaks[0], peaks[1])
#filter out events shorter than 16 ms, or closer than 16 ms to nearest event
timeFilteredEvents = timeFilterEvents(trace, prelimEvents)
#plot time filtered events
plot_prelimEvents_withfilter(trace, covar_AB, prelimEvents)

In [ ]:
#save the events as a numpy array containing the bead positions
#also, save a numpy array containing the force (calculated from displacement of motor bead) and the lifetime
save_events_isoFB(trace, timeFilteredEvents, covar_AB)

In [ ]:
#load event force_dur arrays
Data_forcedur = pd.read_csv('/Users/bob/Library/CloudStorage/Box-Box/4M493IcHMM_isometricFB/20230905/M493I_durvsforce.txt').to_numpy(dtype=np.float32)
#convert event lifetimes to detachment rates by inverting the values
Data_forcedur_kdetach = 1/Data_forcedur[:,1]
#regenerate the original array as force and detachment rate
Data_forcedur_inv = np.column_stack((Data_forcedur[:,0], Data_forcedur_kdetach))

In [ ]:


#Steps for filtering data to select for forces within dynamic range
data = Data_forcedur_inv

# Step 1: Filter data to exclude forces outside a desired range (e.g., 5 to 25)
force_threshold_min = 0
force_threshold_max = 10
filtered_data = data[(data[:, 0] >= force_threshold_min) & (data[:, 0] <= force_threshold_max)]

# Step 2: Define the bins for the force measurements (for example, using np.histogram_bin_edges)
bins = np.linspace(force_threshold_min, force_threshold_max, 20)  # You can adjust the number of bins

# Step 3: Bin the forces and link to corresponding lifetime measurements
bin_indices = np.digitize(filtered_data[:, 0], bins)

# Initialize lists to store averages for each bin
Data_force_bin_averages = []
Data_lifetime_bin_averages = []
Data_force_std = []
Data_lifetime_std = []

# Step 4: Loop through each bin and calculate the average force and lifetime for data in that bin
for i in range(1, len(bins)):
    # Select the data in the current bin
    in_bin = filtered_data[bin_indices == i]
    
    if len(in_bin) > 0:  # Only if there's data in this bin
        # Calculate the average force and lifetime for this bin
        avg_force = np.mean(in_bin[:, 0])
        avg_lifetime = np.mean(in_bin[:, 1])
        # Calculate standard deviations
        std_force = np.std(in_bin[:, 0])
        std_inv_lifetime = np.std(1 / in_bin[:, 1])
        
        # Store the averages
        Data_force_bin_averages.append(avg_force)
        Data_lifetime_bin_averages.append(avg_lifetime)
        Data_force_std.append(std_force)
        Data_lifetime_std.append(std_inv_lifetime)

In [ ]:
#sample plotting, data names and values must be changed!
plt.errorbar(WT_force_bin_averages, WT_lifetime_bin_averages, 
             xerr=WT_force_std, yerr=WT_lifetime_std, fmt='o', color='blue',
             ecolor='black', elinewidth=1.5, capsize=4, alpha=0.75)
plt.errorbar(M493I_force_bin_averages, M493I_lifetime_bin_averages, 
             xerr=M493I_force_std, yerr=M493I_lifetime_std, fmt='o', color='mediumspringgreen',
             ecolor='black', elinewidth=1.5, capsize=4, alpha = 0.75)
plt.plot(BE_forces, bell_equation(BE_forces, WT_k0, WT_d), color='darkblue')
plt.plot(BE_forces, bell_equation(BE_forces, M493I_k0, M493I_d), color='darkgreen')
plt.fill_between(BE_forces, bell_equation(BE_forces, 52.43, 0.5), bell_equation(BE_forces, 41.01, 1.05), color='skyblue', alpha=0.25)
plt.fill_between(BE_forces, bell_equation(BE_forces, 27.54, 0.27), bell_equation(BE_forces, 22.378, 0.5), color='mediumspringgreen', alpha=0.5)
#plt.savefig('/Users/bob/Desktop/M493I figures/IsometricFBlogscalebinnedwCIanderror6.svg', format='svg')
plt.show()